In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read in the Data

In [ ]:
import pandas as pd
import glob
import os
import numpy as np

# Full NCAA Dataset - 48 sec
df = pd.read_csv("/content/drive/Shareddrives/USC Baseball/Datasets/Full_22-24_NCAA_Dataset.csv")

<ipython-input-2-fd0225edde3b>:7: DtypeWarning: Columns (27,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/Shareddrives/USC Baseball/Datasets/Full_22-24_NCAA_Dataset.csv")


In [ ]:
college_extra_info = pd.read_csv("/content/drive/Shareddrives/USC Baseball/Datasets/college_stats_2024_extra_info.csv")

# Clean up the data + filter data + create target column


In [ ]:
#This will ignore all DeprecationWarning warnings in your code.
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
cleandf = df[df['TaggedPitchType'].str.contains("Undefined|Other|,") == False]
cleanD1 = cleandf[cleandf['Level'] == "D1"]

In [ ]:
cleanConf = cleanD1[cleanD1['League'].isin(["SEC", "BIG12", "BIG10", "ACC", "PAC12", "CUSA", "SBELT", "AMER"])]

In [ ]:
# Recodes Pitch Handedness to binary variable

conditions = [
    (cleanConf['PitcherThrows'] == 'Right'),  # RHP
    (cleanConf['PitcherThrows'] == 'Left')  # LHP
]

# creates a list of values to assign for pitch handedness where RHP = 1 & LHP = 0
values = [0, 1]
# creates new column determining pitch hand
cleanConf['PitcherHand'] = np.select(conditions, values)

<ipython-input-7-783810b04da9>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanConf['PitcherHand'] = np.select(conditions, values)


In [ ]:
cleanConf['PitcherHand'].value_counts()

,count
PitcherHand,
0,1290893
1,500407


In [ ]:
cleanConf.shape[0]

1791300

In [ ]:
cleanConf = cleanConf[~((cleanConf['PitchCall'].isin(["InPlay", "FoulBall", "FoulBallNotPlayable"])) & (pd.isna(cleanConf['Angle'])))]

In [ ]:
cleanConf.shape[0]

1769684

In [ ]:
cleanConf.shape[0]

1712724

In [ ]:
conditions = [
    (cleanConf['PitchCall'] == 'StrikeSwinging'),  # Whiff
    (cleanConf['PitchCall'] == 'StrikeCalled'),  # Called Strike
    (cleanConf['Angle'] >= 38), # Pop up launch angle
    (cleanConf['Angle'] <= -5) # Ground ball launch angle
]

values = [1, 1, 1, 1]
cleanConf['isStuffResult'] = np.select(conditions, values, default = 0)

In [ ]:
cleanConf['isStuffResult'].value_counts()

,count
isStuffResult,
0,984068
1,728656


In [ ]:
# Function to convert from "Lastname, Firstname" to "Firstname Lastname"

def convert_name(name):
    parts = name.split(',')
    if len(parts) == 2:
        return parts[1].strip() + ' ' + parts[0].strip()
    else:
        return name

In [ ]:
# Function to read in player height from "college_extra_info"

def get_height(name):
    try:
        # Look up the player by name and get the corresponding height
        height = college_extra_info.loc[college_extra_info['Player'] == name, 'height']

        if not height.empty:
            return height.values[0]  # Return the first (and only) match

        # If no player with the given name is found
        return f"Player '{name}' not found."

    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
# Function that convert player height to shoulder height

def get_shoulder_height(player_height):
  try:
      # Split the height string by "-"
      feet, inches = map(int, player_height.split('-'))

      # Convert inches to decimal feet (1 foot = 12 inches)
      decimal_height = feet + inches / 12

      shoulder_height = 0.7 * decimal_height

      return shoulder_height  # Rounded to 4 decimal places for precision
  except Exception as e:
      return f"Invalid height format: {e}"

In [ ]:
# Function that creates arm angle based on shoulder height, release height, release side

# def get_arm_angle(shoulder_height, release_height, release_side):


In [ ]:
# Function that takes in dataframe, runs through above 4 functions, and creates new column in the dataframe

def process_player_info(df):
  # Apply the functions row by row to calculate the arm angle
    def process_row(row):
        name = row['Pitcher']
        # release_height = row['Rel']
        # release_side = row['release_side']

        # Get player height from info_df
        height_str = get_height(name)

        if height_str:
            # Convert height to decimal format
            shoulder_height = get_shoulder_height(height_str)

            if shoulder_height is not None:
                # Calculate the arm angle
                # arm_angle = get_arm_angle(shoulder_height, release_height, release_side)
                # return arm_angle
                return shoulder_height

        return None  # Return None if any step fails

    # Apply the process_row function to each row and store results in a new 'arm_angle' column
    df['shoulder_height'] = df.apply(process_row, axis=1)

    return df

In [ ]:
# cleanConf = process_player_info(cleanConf)

In [ ]:
print("Clean Conference size: " + str(len(cleanConf)))

Clean Conference size: 1712724


# Making the model

In [ ]:
finalCleanDF = cleanConf.dropna(subset=['isStuffResult', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension'])
finalDF = finalCleanDF[['isStuffResult', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension']]

In [ ]:
finalDF.reset_index()

,index,isStuffResult,PitcherHand,RelSpeed,SpinRate,InducedVertBreak,HorzBreak,SpinAxis,RelHeight,RelSide,Extension
0,0,1,0,99.52724,2694.881660,16.74166,17.86536,228.580072,6.12523,2.49133,5.42371
1,1,0,0,99.97207,2591.952252,14.54879,16.43906,230.378430,6.14564,2.23451,5.55892
2,2,1,0,82.98675,2868.547909,-3.63477,-9.01126,61.314872,6.16553,2.49581,4.63345
3,3,1,0,86.45521,2809.630437,7.70886,-9.07776,126.013235,5.99836,2.49053,4.98830
4,4,0,0,82.26110,2939.165899,-5.50744,-12.05288,60.441938,6.24191,4.11979,4.85250
...,...,...,...,...,...,...,...,...,...,...,...
1701434,3509696,0,0,89.14162,2786.309514,7.40339,-0.91756,171.818667,5.50441,1.93165,4.70968
1701435,3509697,0,0,97.21620,2718.863995,12.05568,15.90280,235.023465,5.28493,2.16669,5.53639
1701436,3509698,1,0,91.19636,2679.303136,9.43819,3.85055,204.529888,5.66796,2.05525,4.93663
1701437,3509699,0,0,89.38379,2761.763969,7.36918,0.88425,187.842951,5.55800,2.09080,5.19417


In [ ]:
print(finalDF.shape)

(1701439, 10)


## Making General Model

In [ ]:
minority_class = finalDF[finalDF['isStuffResult'] == 1]
majority_class = finalDF[finalDF['isStuffResult'] == 0]

print("majority class is: " + str(len(majority_class)) + " (" + str(100*len(majority_class)/(len(majority_class)+len(minority_class))) + ")")
print("minority class is: " + str(len(minority_class)) + " (" + str(100*len(minority_class)/(len(majority_class)+len(minority_class))) + ")")

print("total is: " + str(len(majority_class)+len(minority_class)))

majority class is: 977448 (57.448312869283)
minority class is: 723991 (42.551687130717)
total is: 1701439


In [ ]:
# resamples the class
from sklearn.utils import resample
majority_class_downsampled = resample(majority_class,
                                      replace=False,  # sample without replacement
                                      n_samples=len(minority_class)*1,  # match minority class size
                                      #n_samples = 390000,
                                      random_state=42)  # reproducible results
minority_class_upsampled = resample(minority_class,
                                      replace=True,  # sample without replacement
                                      n_samples=int(len(minority_class)*1),  # match minority class size
                                      #n_samples = 390000,
                                      random_state=42)  # reproducible results

# Combine minority class and downsampled majority class
balanced_df = pd.concat([minority_class_upsampled, majority_class_downsampled])




In [ ]:
# Shuffle the data to avoid order effects
# figure why this don't work
# balanced_df = balanced_df.sample(frac=1, random_state=42)
balanced_df = balanced_df.sample(frac=1, random_state=42)


In [ ]:
rel_data = balanced_df[['isStuffResult', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension']].to_numpy()


classification = balanced_df['isStuffResult'].to_numpy()

In [ ]:
minority = balanced_df[balanced_df['isStuffResult'] == 1]
majority = balanced_df[balanced_df['isStuffResult'] != 1]

print("majority class is: " + str(len(majority)))
print("minority class is: " + str(len(minority)))

print("total is: " + str(len(majority)+len(minority)))

sampledTotal = len(majority) + len(minority)
trainTotal = int(0.8*sampledTotal)

majority class is: 723991
minority class is: 723991
total is: 1447982


In [ ]:
# Split data into features and target
X = balanced_df.drop(['isStuffResult'], axis=1)
y = balanced_df['isStuffResult']

# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
# X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# print(f"Train size: {len(X_train)}")
# print(f"Test size: {len(X_test)}")
# print(f"Validation size: {len(X_val)}")

# Split data into training and testing sets
x_train = X[:trainTotal]
x_test = X[trainTotal:]
y_train = y[:trainTotal]
y_test = y[trainTotal:]

from sklearn import preprocessing

# Standardize the features
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### Using Optuna Hyperparam Tuning

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.0 MB/s eta 0:00:00


In [ ]:
import optuna
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Define objective function for Optuna
def objective(trial):
    # Suggest hyperparameters to tune
    eta = trial.suggest_float('eta', 0.0001, 0.1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 150)
    n_estimators = trial.suggest_int('n_estimators', 50, 200)

    # Standardize the data
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    # Convert the data into DMatrix for GPU processing
    dtrain = xgb.DMatrix(x_train_scaled, label=y_train)
    dtest = xgb.DMatrix(x_test_scaled, label=y_test)

    # Set parameters for GPU usage and training
    params = {
        'eta': eta,
        'max_depth': max_depth,
        'objective': 'binary:logistic',  # Assuming binary classification
        'eval_metric': 'logloss',
        'tree_method': 'hist',       # Use GPU for training
        'device': 'cuda',    # Use GPU for prediction
    }

    # Train the model using DMatrix and suggested hyperparameters
    model = xgb.train(params, dtrain, num_boost_round=n_estimators)

    # Predict using the test set
    y_pred_proba = model.predict(dtest)
    y_pred = [1 if p > 0.5 else 0 for p in y_pred_proba]  # Convert probabilities to binary predictions

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Print trial result
    print(f"Trial {trial.number}: Accuracy={accuracy:.4f}, eta={eta}, max_depth={max_depth}, n_estimators={n_estimators}")

    return accuracy  # Optuna will try to maximize this

# Create a study object and specify to maximize accuracy
study = optuna.create_study(direction='maximize')

# Start optimization (define the number of trials)
study.optimize(objective, n_trials=20)

# Get the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print(f"  Best hyperparameters: {trial.params}")


[I 2024-09-16 19:16:07,515] A new study created in memory with name: no-name-7736011e-2114-4511-b707-ff90b29b1e4a
<ipython-input-33-b93668afb479>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 0.0001, 0.1)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:16:07] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:16:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.

Trial 0: Accuracy=0.6584, eta=0.003241898172191929, max_depth=100, n_estimators=70


<ipython-input-33-b93668afb479>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 0.0001, 0.1)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:19:10] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:19:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:21:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is depr

Trial 1: Accuracy=0.6622, eta=0.005226151243909066, max_depth=24, n_estimators=134


<ipython-input-33-b93668afb479>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 0.0001, 0.1)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:21:31] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:21:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:27:14] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is depr

...

Trial 17: Accuracy=0.6201, eta=0.0002447706960966429, max_depth=91, n_estimators=53


<ipython-input-33-b93668afb479>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 0.0001, 0.1)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:26:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:26:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:29:40] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is depr

Trial 18: Accuracy=0.7101, eta=0.04726663419818292, max_depth=53, n_estimators=97


<ipython-input-33-b93668afb479>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 0.0001, 0.1)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:29:42] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:29:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:34:03] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is depr

Trial 19: Accuracy=0.6980, eta=0.010191378560412401, max_depth=102, n_estimators=101
Best trial:
  Accuracy: 0.7111192450198034
  Best hyperparameters: {'eta': 0.08387896876501985, 'max_depth': 66, 'n_estimators': 109}


### Making Stuff+ Model

In [ ]:
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.ensemble import RandomForestClassifier
from numpy import loadtxt
from xgboost import XGBClassifier
# from cuml.ensemble import RandomForestClassifier as cuRF
# !pip install -U xgboost

names = [
    "Random Forest",
    "XGBoost"
]

classifiers = [
    # RandomForestClassifier(max_depth=None, n_estimators=100),
    XGBClassifier(device='gpu', eta=0.07, max_depth=66, n_estimators=110, eval_metric='logloss'),
    # XGBClassifier(device='gpu', eta=0.08, max_depth=66, n_estimators=110, eval_metric='logloss')
]

In [ ]:
accuracies = []
for opt in classifiers:
  clf = make_pipeline(StandardScaler(), opt)
  clf.fit(x_train, y_train)

  y_pred = clf.predict((x_test))
  accuracies.append(metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
target_names = ['not ideal, yes ideal']
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.74      0.72    144984
           1       0.72      0.69      0.70    144613

    accuracy                           0.71    289597
   macro avg       0.71      0.71      0.71    289597
weighted avg       0.71      0.71      0.71    289597



In [ ]:
print(accuracies)

[0.7119825136310114, 0.7104320832052818]


In [ ]:
print(accuracies)

[0.7119825136310114]


### Export Stuff+ Model

In [ ]:
import joblib
from joblib import dump

import os
os.chdir('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Stuff+')

# export the trained model to disk
joblib.dump(clf, 'GeneralStuffModel.joblib')

['GeneralStuffModel.joblib']